## Import Librabries

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier  


## Step 1: Load and Explore the Dataset

In [8]:
data = pd.read_csv("Alphabets_data.csv")
print("Dataset Shape:", data.shape)
print("First 5 Rows:\n", data.head())
print("Dataset Info:\n")
data.info()
print("\nMissing Values:\n", data.isnull().sum())


Dataset Shape: (20000, 17)
First 5 Rows:
   letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
Dataset Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  ----------

## Step 2: Data Preprocessing

In [10]:
# Separate features and target variable
X = data.iloc[:, 1:]  # Features
y = data['letter']    # Target variable


In [11]:
# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [12]:
# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [13]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)


## Step 3: Define the Basic ANN Model

In [15]:
def create_model(activation='relu', optimizer='adam'):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Updated to use Input layer for clarity
        Dense(128, activation=activation),  # First hidden layer
        Dense(64, activation=activation),  # Second hidden layer
        Dense(len(np.unique(y_encoded)), activation='softmax')  # Output layer
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# Callback for early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Instantiate and train the basic model
basic_model = create_model()
history = basic_model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32,callbacks=[early_stopping])

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4404 - loss: 2.0806 - val_accuracy: 0.7878 - val_loss: 0.7764
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7894 - loss: 0.7097 - val_accuracy: 0.8418 - val_loss: 0.5559
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8484 - loss: 0.5232 - val_accuracy: 0.8765 - val_loss: 0.4418
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8796 - loss: 0.4190 - val_accuracy: 0.8930 - val_loss: 0.3698
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8984 - loss: 0.3465 - val_accuracy: 0.9035 - val_loss: 0.3290
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9093 - loss: 0.3031 - val_accuracy: 0.9107 - val_loss: 0.2982
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9274 - loss: 0.2537 - val_accuracy: 0.9227 - val_loss: 0.2693
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9350 - loss: 0.2204 - val_accuracy: 0.

## Step 5: Hyperparameter Tuning using GridSearchCV

In [ ]:
def build_model(activation='relu', optimizer='adam'):
    return create_model(activation=activation, optimizer=optimizer)

# Updated KerasClassifier initialization to resolve the '-sklearn-tags-' error
model = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [20, 50],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'sgd']
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2, scoring='accuracy', error_score='raise')
grid_result = grid.fit(X_train, y_train)
